In [10]:
import string
from datetime import datetime
import os
import numpy as np
import pandas as pd
import json
from collections import defaultdict
from Queue import Queue


import nltk
# nltk.data.path.append("/")

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from gensim.models.word2vec import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.doc2vec import TaggedDocument
from gensim.models.doc2vec import Doc2Vec

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import plotly
plotly.tools.set_credentials_file(username='amcire96', api_key='sej35ud4YbSOfIshhhZg')
# print(plotly.__version__)

import plotly.plotly as py
from plotly.graph_objs import *
import requests
requests.packages.urllib3.disable_warnings()

In [3]:
reviews = []
with open("yelp_academic_dataset_review.json", "r") as f:
    for line in f:
        json_line = json.loads(line)
        reviews.append(json_line)

In [4]:
print(len(reviews))
reviews[0]

4153150


{u'business_id': u'2aFiy99vNLklCx3T_tGS9A',
 u'cool': 0,
 u'date': u'2011-10-10',
 u'funny': 0,
 u'review_id': u'NxL8SIC5yqOdnlXCg18IBg',
 u'stars': 5,
 u'text': u"If you enjoy service by someone who is as competent as he is personable, I would recommend Corey Kaplan highly. The time he has spent here has been very productive and working with him educational and enjoyable. I hope not to need him again (though this is highly unlikely) but knowing he is there if I do is very nice. By the way, I'm not from El Centro, CA. but Scottsdale, AZ.",
 u'type': u'review',
 u'useful': 0,
 u'user_id': u'KpkOkG6RIf4Ra25Lhhxf1A'}

In [5]:
businesses = []
with open("yelp_academic_dataset_business.json", "r") as f:
    for line in f:
        json_line = json.loads(line)
        businesses.append(json_line)

In [6]:
print(len(businesses))
businesses[0]

144072


{u'address': u'227 E Baseline Rd, Ste J2',
 u'attributes': [u'BikeParking: True',
  u'BusinessAcceptsBitcoin: False',
  u'BusinessAcceptsCreditCards: True',
  u"BusinessParking: {'garage': False, 'street': False, 'validated': False, 'lot': True, 'valet': False}",
  u'DogsAllowed: False',
  u'RestaurantsPriceRange2: 2',
  u'WheelchairAccessible: True'],
 u'business_id': u'0DI8Dt2PJp07XkVvIElIcQ',
 u'categories': [u'Tobacco Shops', u'Nightlife', u'Vape Shops', u'Shopping'],
 u'city': u'Tempe',
 u'hours': [u'Monday 11:0-21:0',
  u'Tuesday 11:0-21:0',
  u'Wednesday 11:0-21:0',
  u'Thursday 11:0-21:0',
  u'Friday 11:0-22:0',
  u'Saturday 10:0-22:0',
  u'Sunday 11:0-18:0'],
 u'is_open': 0,
 u'latitude': 33.3782141,
 u'longitude': -111.936102,
 u'name': u'Innovative Vapors',
 u'neighborhood': u'',
 u'postal_code': u'85283',
 u'review_count': 17,
 u'stars': 4.5,
 u'state': u'AZ',
 u'type': u'business'}

In [7]:
valid_business_ids = set()
business_id_to_index = {}
business_index_to_bid = {}
c = 0
for business in businesses:
    if business["review_count"] >= 1000:
        valid_business_ids.add(business["business_id"])
        if business["business_id"] not in business_id_to_index:
            business_id_to_index[business["business_id"]] = c
            business_index_to_bid[c] = business["business_id"]
            c += 1
print(len(valid_business_ids))
valid_business_ids

177


{u'--9e1ONYQuAa-CB_Rrw7Tw',
 u'-ed0Yc9on37RoIoG2ZgxBA',
 u'0FUtlsQrJI7LhqDPxLumEw',
 u'0NmTwqYEQiKErDv4a55obg',
 u'0W4lkclzZThpx3V65bVgig',
 u'2iTsRqUsPGRH1li1WVRvKQ',
 u'2weQS-RnoOBhb1KsHKyoSQ',
 u'3BCsAgo_1i4xMuTyLKMLRQ',
 u'3GEEy7RP6e4bT4LAiWFMFQ',
 u'3N9U549Zse8UP-MwKZAjAQ',
 u'3kdSl5mo9dWC4clrQjEDGg',
 u'3l54GTr8-E3XPbIxnF_sAA',
 u'3xmfT7l3xNH5LK1dLzfvGw',
 u'4GXII-GU7S0ZyU6ElkhscQ',
 u'4JNXUYY8wbaaDmk3BPzlWw',
 u'4k3RlMAMd46DZ_JyZU0lMg',
 u'5LNZ67Yw9RD6nf4_UhXOjw',
 u'5T6kFKFycym_GkhgOiysIw',
 u'5iSmZO0SrKU6EoXK_1M8Kw',
 u'5shgJB7a-2_gdnzc0gsOtg',
 u'6Ct57qgmXwOnzfSZoUGh0Q',
 u'6Q7-wkCPc1KF75jZLOTcMw',
 u'7fxebHYUwIF6CakxSr70iQ',
 u'7sPNbCx7vGAaH7SbNPZ6oA',
 u'7sb2FYLS2sejZKxRYF9mtg',
 u'9SU7ZZhaFUJJ6m2k5HKHeg',
 u'9a3DrZvpYxVs3k_qwlCNSw',
 u'A-uZAD4zP3rRxb44WUGV5w',
 u'A5Rkh7UymKm0_Rxm9K2PJw',
 u'AV6weBrZFFBfRGCbcRGO4g',
 u'Az_60nNuh1FH8Ds8oasZjw',
 u'BH9z7IJ4zydAqgwsbqoVZQ',
 u'BIBWGO_r_1znnlmLbp4Nxg',
 u'BLIJ-p5wYuAhw6Pp6mh6mw',
 u'ByFMv3p5X1aNeZhU61rDcA',
 u'CiYLq33nAyghFkUR1

In [8]:
only_text = []
ordered_review_ids = []
ordered_business_id_indices = []
bid_to_rid_dict = defaultdict(list)
rid_to_bid_dict = {}
for review in reviews:
    if review["business_id"] in valid_business_ids:
        only_text.append(review["text"])
        ordered_review_ids.append(review["review_id"])
        ordered_business_id_indices.append(business_id_to_index[review["business_id"]])
        bid_to_rid_dict[review["business_id"]].append(review["review_id"])
        rid_to_bid_dict[review["review_id"]] = review["business_id"]
print(len(only_text))
print(len(ordered_review_ids))

315002
315002


In [10]:

tfidf_vectorizer = TfidfVectorizer(input=only_text, lowercase=True, stop_words="english", max_df=0.9, min_df=100)
tfidf_review_text = tfidf_vectorizer.fit_transform(only_text)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

print(tfidf_review_text.shape)

(315002, 9071)


In [11]:
count_vectorizer = CountVectorizer(input=only_text, lowercase=True, stop_words="english", max_df=0.9, min_df=100)
count_review_text = count_vectorizer.fit_transform(only_text)
count_feature_names = count_vectorizer.get_feature_names()

print(count_review_text.shape)

(315002, 9071)


In [12]:
print(count_feature_names)

[u'00', u'000', u'00am', u'00pm', u'05', u'07', u'08', u'09', u'10', u'100', u'1000', u'101', u'1030', u'105', u'108', u'109', u'10am', u'10min', u'10oz', u'10pm', u'10th', u'10x', u'11', u'110', u'115', u'11am', u'11pm', u'11th', u'12', u'120', u'1200', u'125', u'12am', u'12oz', u'12pm', u'12th', u'13', u'130', u'14', u'140', u'14th', u'15', u'150', u'1500', u'15min', u'15mins', u'15pm', u'15th', u'16', u'160', u'16oz', u'16th', u'17', u'170', u'1762', u'17th', u'18', u'180', u'18th', u'19', u'19th', u'1am', u'1hr', u'1pm', u'1st', u'20', u'200', u'2000', u'2005', u'2006', u'2007', u'2008', u'2009', u'2010', u'2011', u'2012', u'2013', u'2014', u'2015', u'2016', u'20min', u'20mins', u'20s', u'20th', u'21', u'21st', u'22', u'22nd', u'23', u'230', u'23rd', u'24', u'24hr', u'24hrs', u'24th', u'25', u'250', u'25th', u'26', u'26th', u'27', u'27th', u'28', u'28th', u'29', u'29th', u'2am', u'2nd', u'2pm', u'2x', u'30', u'300', u'3000', u'30am', u'30min', u'30mins', u'30pm', u'30s', u'30th', u

In [13]:
def display_topics(model, feature_names, n_top_words):
    for topic_index, topic in enumerate(model.components_):
        print("Topic %d:" % topic_index)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words:-1]]))

In [14]:
lda_model = LatentDirichletAllocation(n_topics=30).fit(count_review_text)
display_topics(lda_model, count_feature_names, 20)

/home/amcire/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning:

The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.



Topic 0:
experience server ate drink bar point table did came plate drinks staff waiter restaurant thank meal boyfriend birthday time
Topic 1:
bathroom bed clean suite hotel resort mgm view loud fee security work charge staff bone smell dirty money large
Topic 2:
years mexican year shows kids ago used times feel like ami couple closed gabi family bay mandalay work saw
Topic 3:
pool water tv non love day rock beautiful watch palazzo gorgeous upgraded tub space big light nicer awesome im
Topic 4:
room check free night stay floor time shower service door did card walk booked didn stayed morning early lobby
Topic 5:
good chicken steak ordered food really just like fried got try order rice didn eat wasn menu soup salad
Topic 6:
staying wife star short street bottle 100 weekend doesn pay drink pad water seriously guy face guys bars cheap
Topic 7:
buffet crab food sushi desk good buffets dinner legs selection quality price lunch eat desserts rib fresh shrimp rolls
Topic 8:
strip vegas better 

In [15]:
nmf_model = NMF(n_components=30).fit(tfidf_review_text)
display_topics(nmf_model, tfidf_feature_names, 20)

Topic 0:
just like don time didn know got said people did think say going bad wasn way come want make
Topic 1:
room check desk bed stay bathroom view shower suite stayed floor night booked service tv called tower checked day
Topic 2:
burger fries burgers shake truffle cheese sweet onion potato bun rings shack ordered bacon medium shakes gordon farm cooked
Topic 3:
great service atmosphere beer selection experience location fun spot view prices happy fantastic wonderful time patio awesome ambiance wine
Topic 4:
buffet crab buffets legs selection seafood desserts prime wynn quality dessert rib station price dinner variety bellagio food wicked
Topic 5:
place try eat looking packed friends cool fun strip want places menu busy town small people cheap beer reviews
Topic 6:
wait line long worth minutes hour waiting 30 waited seated time hours table 45 15 20 10 check lines
Topic 7:
good pretty price overall beer little selection decent bit prices ok thing pricey location bad value got taste wa

In [16]:
tagged_doc_list = []
for i in range(len(only_text)):
    text = only_text[i]
    tokens = nltk.word_tokenize(text)
    label = ordered_business_id_indices[i]
    tagged_doc = TaggedDocument(tokens, [label])
    tagged_doc_list.append(tagged_doc)

In [17]:
tagged_doc_list[0]

TaggedDocument(words=[u'I', u'love', u'sushi', u'!', u'Especially', u'with', u'cream', u'cheese', u'!', u'This', u'place', u'hardly', u'had', u'any', u'cream', u'cheese', u'sushi', u'but', u'it', u'is', u'still', u'overall', u'YUM', u'!', u':', u')', u'Service', u'is', u'excellent', u',', u'you', u'never', u'go', u'thirsty', u'as', u'they', u'keep', u'your', u'drink', u'constantly', u'full', u'!', u'Dinner', u'AYCE', u'is', u'around', u'$', u'26', u',', u'while', u'I', u"'ve", u'seen', u'lower', u'prices', u'.', u'I', u'tried', u'the', u'Philadelphia', u'roll', u',', u'caterpillar', u'and', u'Dragon', u'roll', u'.', u'philadelphia', u'roll', u'for', u'some', u'reason', u'was', u"n't", u'as', u'tasty', u'as', u'I', u'thought', u'.', u'Any', u'eel', u'in', u'the', u'rolls', u'I', u'have', u'tried', u'are', u'not', u'as', u'good', u'to', u'brag', u'about', u'honestly', u'.', u'I', u'give', u'4', u'stars', u'for', u'the', u'great', u'service', u',', u'and', u'it', u"'s", u'enough', u'to', 

In [18]:
only_text[0]

u"I love sushi! Especially with cream cheese! This place hardly had any cream cheese sushi but it is still overall YUM! :) Service is excellent, you never go thirsty as they keep your drink constantly full!\n\nDinner AYCE is around $26, while I've seen lower prices. I tried the Philadelphia roll, caterpillar and Dragon roll. philadelphia roll for some reason wasn't as tasty as I thought. Any eel in the rolls I have tried are not as good to brag about honestly. I give 4 stars for the great service, and it's enough to go back and give them another chance."

In [19]:
model = Doc2Vec(size=500, min_count=100, iter=5)
model.build_vocab(tagged_doc_list)
%time model.train(tagged_doc_list)

CPU times: user 24min 9s, sys: 20.7 s, total: 24min 30s
Wall time: 8min 54s


171048124

In [20]:
model.save('reviews_doc2vec_model.doc2vec')


In [9]:
# load the model back
model = Doc2Vec.load('reviews_doc2vec_model.doc2vec')

In [10]:
example_index = 100

In [11]:
example_bid = rid_to_bid_dict[ordered_review_ids[example_index]]
example_rids_for_business = bid_to_rid_dict[example_bid]

In [24]:
print(len(example_rids_for_business))
example_bid

1502


u'JyxHvtj-syke7m9rbza7mA'

In [25]:

print("Review Text is:")
print(only_text[example_index])

for rid in example_rids_for_business[:5]:
    ind = ordered_review_ids.index(rid)
    print
    print("---------------")
    print("Review index %d is:" % ind)
    print(only_text[ind])

Review Text is:
Have been coming here for past year or so
great AYCE sushi place not too far from where I live

used to frequent Sushi Factory, one of the first AYCE places in town, but sadly they closed :-(

in any case, have tried a few other sushi places, have found this one to be one of the better choices in the area

usually pretty busy, never been there without having to wait...boo

their selection is pretty good, fish is fresh, haven't really had any bad meals to date
I usually try to sit at sushi bar unless with more than a couple friends, as you can usually get some good choices/tips from sushi chefs working that day

I usually stick to nigiri, so not a whole lot to explain other that the fish is usually fresh, and the ratio to rice pretty good too....make sure to check out board for specials!

love that they have salmon collar too! I do order a few of the cooked items, such as that, maybe some of the grilled specials, and oh yeah, pork belly, YUM!

I like their tuna on crispy

In [26]:
inferred = model.infer_vector(nltk.word_tokenize(only_text[100]))
inferred

array([ -1.29789323e-01,  -1.72013208e-01,   4.36630487e-01,
         3.33004355e-01,   6.95068419e-01,  -9.19543505e-02,
         3.10375363e-01,   4.24267262e-01,   1.07282199e-01,
         3.65397871e-01,  -2.36524139e-02,  -5.43285310e-01,
        -6.01791628e-02,  -7.96233177e-01,  -1.48960292e-01,
         9.65624824e-02,   2.62469530e-01,   3.68752122e-01,
        -5.34154475e-02,   1.93226665e-01,  -3.29037845e-01,
         3.91689956e-01,  -1.93448052e-01,  -2.97535565e-02,
         1.62627563e-01,   2.22769994e-02,   2.41268352e-01,
         4.43348765e-01,  -3.01022351e-01,   1.50144786e-01,
         3.30046982e-01,   2.33760133e-01,  -4.32248749e-02,
         2.18418211e-01,   1.06368206e-01,  -2.76062161e-01,
         1.30623728e-01,  -9.72787887e-02,   2.12684855e-01,
         2.63790965e-01,  -6.80630803e-02,  -2.93577254e-01,
         1.92500442e-01,   3.29926498e-02,  -2.23459110e-01,
         1.35441631e-01,   3.98774803e-01,  -1.06399751e+00,
         7.20194429e-02,

In [27]:
sims = model.docvecs.most_similar([inferred], topn=len(model.docvecs))
sims

[(127, 0.5033470392227173),
 (141, 0.4891008138656616),
 (135, 0.4690977931022644),
 (140, 0.44080260396003723),
 (35, 0.4326072931289673),
 (32, 0.4120200276374817),
 (134, 0.3679501414299011),
 (128, 0.36253756284713745),
 (3, 0.3579981029033661),
 (148, 0.35294073820114136),
 (170, 0.34191229939460754),
 (29, 0.3299570083618164),
 (78, 0.32656335830688477),
 (30, 0.32073163986206055),
 (37, 0.3182520866394043),
 (147, 0.29989930987358093),
 (150, 0.2987726032733917),
 (75, 0.29649096727371216),
 (133, 0.28910961747169495),
 (8, 0.28702253103256226),
 (47, 0.2863979935646057),
 (146, 0.28428345918655396),
 (99, 0.2819390296936035),
 (165, 0.2788647413253784),
 (159, 0.2712439298629761),
 (155, 0.26929527521133423),
 (154, 0.2652243673801422),
 (104, 0.2632179856300354),
 (84, 0.2610851526260376),
 (142, 0.25292685627937317),
 (81, 0.24877353012561798),
 (7, 0.24821865558624268),
 (130, 0.24625833332538605),
 (101, 0.24521979689598083),
 (28, 0.24444738030433655),
 (125, 0.24226815998

In [13]:
def compute_similarities(start_ind, end_ind):
    distances = []
    c = 0.0
    sum_vectors = np.zeros(500)
    for i in range(start_ind, end_ind+1):
        
        text = only_text[i]
        word2vec_vector = model.infer_vector(nltk.word_tokenize(text))
        curr_business_vector = sum_vectors / max(1.0, c)
        distances.append(cosine_similarity(word2vec_vector.reshape(1,-1), curr_business_vector.reshape(1,-1))[0,0])
        sum_vectors += word2vec_vector
        c += 1
    return distances

In [14]:
bid_to_min_ind = {}
bid_to_max_ind = {}
for i, business_index in enumerate(ordered_business_id_indices):
    if business_index_to_bid[business_index] not in bid_to_min_ind:
        bid_to_min_ind[business_index_to_bid[business_index]] = i
    bid_to_max_ind[business_index_to_bid[business_index]] = i


In [15]:
bid_to_min_ind

{u'--9e1ONYQuAa-CB_Rrw7Tw': 162659,
 u'-ed0Yc9on37RoIoG2ZgxBA': 311746,
 u'0FUtlsQrJI7LhqDPxLumEw': 185134,
 u'0NmTwqYEQiKErDv4a55obg': 266425,
 u'0W4lkclzZThpx3V65bVgig': 91336,
 u'2iTsRqUsPGRH1li1WVRvKQ': 1502,
 u'2weQS-RnoOBhb1KsHKyoSQ': 62516,
 u'3BCsAgo_1i4xMuTyLKMLRQ': 99477,
 u'3GEEy7RP6e4bT4LAiWFMFQ': 125424,
 u'3N9U549Zse8UP-MwKZAjAQ': 176635,
 u'3kdSl5mo9dWC4clrQjEDGg': 48979,
 u'3l54GTr8-E3XPbIxnF_sAA': 43724,
 u'3xmfT7l3xNH5LK1dLzfvGw': 152067,
 u'4GXII-GU7S0ZyU6ElkhscQ': 123123,
 u'4JNXUYY8wbaaDmk3BPzlWw': 272498,
 u'4k3RlMAMd46DZ_JyZU0lMg': 143635,
 u'5LNZ67Yw9RD6nf4_UhXOjw': 112054,
 u'5T6kFKFycym_GkhgOiysIw': 263843,
 u'5iSmZO0SrKU6EoXK_1M8Kw': 309666,
 u'5shgJB7a-2_gdnzc0gsOtg': 102505,
 u'6Ct57qgmXwOnzfSZoUGh0Q': 172709,
 u'6Q7-wkCPc1KF75jZLOTcMw': 38324,
 u'7fxebHYUwIF6CakxSr70iQ': 31192,
 u'7sPNbCx7vGAaH7SbNPZ6oA': 56147,
 u'7sb2FYLS2sejZKxRYF9mtg': 218608,
 u'9SU7ZZhaFUJJ6m2k5HKHeg': 163970,
 u'9a3DrZvpYxVs3k_qwlCNSw': 33578,
 u'A-uZAD4zP3rRxb44WUGV5w': 242479,
 u'

In [16]:
example_bid_index = 1
example_bid = business_index_to_bid[example_bid_index]

In [17]:
similarities = compute_similarities(bid_to_min_ind[example_bid], bid_to_max_ind[example_bid])
similarities

[0.0,
 0.47163118074590005,
 0.51629162813547413,
 0.71817429102446861,
 0.69105113621074554,
 0.73547395650621072,
 0.74355244655696007,
 0.80761828985000794,
 0.59103352965763389,
 0.66712321011183695,
 0.78646094739624006,
 0.81532410921100273,
 0.7777862946407923,
 0.57974712717639831,
 0.68562013002491096,
 0.8050695032628512,
 0.68331750334844688,
 0.64009326441424175,
 0.74470672440422092,
 0.64020757184628141,
 0.75425458154925051,
 0.79209252115028128,
 0.77037843976408582,
 0.64691463691346396,
 0.62380982362165616,
 0.82651394330303352,
 0.8161901997733676,
 0.64988852643517181,
 0.70345811601672303,
 0.77413658933012464,
 0.67529717898799868,
 0.67137687890043229,
 0.64910239269577186,
 0.70807100587898053,
 0.73489389149021456,
 0.69659910789432811,
 0.73153933682004824,
 0.58170019011218499,
 0.76595069540714023,
 0.8147577371407968,
 0.83383874120122425,
 0.74498254787574969,
 0.75529993828339581,
 0.69676805133937347,
 0.83547971274270028,
 0.79791104077522501,
 0.77895

In [18]:
# Create a trace
trace = Scatter(
    x = range(len(similarities)),
    y = similarities
)

data = [trace]

py.iplot(data)

In [19]:
def compute_similarities_sliding_window(start_ind, end_ind, window_size=10):
    distances = []
    sum_vectors = np.zeros(500)
    vectors_in_window = Queue(maxsize=window_size)
    for i in range(start_ind, end_ind+1):        
#         print(vectors_in_window.qsize())
        text = only_text[i]
        word2vec_vector = model.infer_vector(nltk.word_tokenize(text))
        curr_business_vector = sum_vectors / max(vectors_in_window.qsize(), 1.0)
        distances.append(cosine_similarity(word2vec_vector.reshape(1,-1), curr_business_vector.reshape(1,-1))[0,0])
        if vectors_in_window.qsize() == window_size:
            popped_vector = vectors_in_window.get()
            sum_vectors -= popped_vector
        vectors_in_window.put(word2vec_vector)
        sum_vectors += word2vec_vector
        
    return distances

In [20]:
similarities_sliding_window = compute_similarities_sliding_window(bid_to_min_ind[example_bid], bid_to_max_ind[example_bid], window_size=10)
similarities_sliding_window

[0.0,
 0.56056228062076308,
 0.48530049382617813,
 0.67116351050441425,
 0.6975254429021065,
 0.749618867595511,
 0.73125063837568038,
 0.7676786370230726,
 0.63027293701291853,
 0.70060514240256355,
 0.76951394445006205,
 0.76545270445139335,
 0.78700085313447399,
 0.57268559960384757,
 0.70686350981855373,
 0.79406815504939265,
 0.67474090026216915,
 0.65045405753433361,
 0.74543127616142002,
 0.64604060787666984,
 0.73164076318954741,
 0.80512501773129008,
 0.72872940808569908,
 0.64222623792578704,
 0.62498204402608692,
 0.79849532483510766,
 0.71810382854452581,
 0.63888643165440939,
 0.66544016756296409,
 0.7016547144092905,
 0.75470826281918013,
 0.63408907990316121,
 0.63039244491623447,
 0.6502112388027449,
 0.68265346354999634,
 0.67081350949398622,
 0.7201094022758272,
 0.52098449067527985,
 0.74270142341977785,
 0.78638445742962282,
 0.79449679162381015,
 0.69254949180353209,
 0.73182397291810897,
 0.69287090512487226,
 0.81015730426885957,
 0.76194338207276568,
 0.75866937

In [21]:
# Create a trace
trace = Scatter(
    x = range(len(similarities_sliding_window)),
    y = similarities_sliding_window
)

data = [trace]

py.iplot(data)

In [22]:
similarities_sliding_window_small = compute_similarities_sliding_window(bid_to_min_ind[example_bid], bid_to_max_ind[example_bid], window_size=5)
similarities_sliding_window_small

[0.0,
 0.5281786362382298,
 0.52562437545046203,
 0.62747546096594331,
 0.68111283754459873,
 0.72760514504345375,
 0.7318591165754309,
 0.75098233430769223,
 0.63305479396179287,
 0.6879379935325769,
 0.73314111393017622,
 0.73200781281899885,
 0.78258693723241779,
 0.53640374416533954,
 0.62886771460294044,
 0.79876080066813393,
 0.66430499237552532,
 0.59537138233920839,
 0.70297399598285404,
 0.5831807928971523,
 0.68947371329855445,
 0.76193677494303502,
 0.69869314157042328,
 0.60753192436727566,
 0.64387681084241333,
 0.76135767897930706,
 0.73864971324349527,
 0.63881237303664817,
 0.62318984216390572,
 0.73564898705516402,
 0.72477061059542136,
 0.62098890764507142,
 0.61628054053913472,
 0.59398410127128054,
 0.64614137292220364,
 0.60875199953677628,
 0.66112679954761855,
 0.57194959402110723,
 0.67204927484583332,
 0.80231465067757124,
 0.79097367467204172,
 0.71343609861597912,
 0.63537569370170499,
 0.7053852325055352,
 0.82316227641158557,
 0.73394058740413182,
 0.716825

In [23]:
# Create a trace
trace = Scatter(
    x = range(len(similarities_sliding_window_small)),
    y = similarities_sliding_window_small
)

data = [trace]

py.iplot(data)